# SGDRegressor 

Nessa regressão foi previsto o impacto das notas (de críticos, usuários e Vgchartzscore) e também do genero do jogo (ação, tiro, aventura, ect) nas vendas totais desse jogo, assim prevendo qual vai ser o proximo genenro mais vendido e quais serão suas notas.

In [394]:
%matplotlib notebook
%reset -f


import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import statsmodels.api as sm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn import preprocessing
from scipy.stats import norm
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

**Importando o dataset**

In [395]:
df = pd.read_excel("jogo.xlsx")

In [396]:
#Tirando os nan.
df = df.fillna(0)

In [397]:
#removando os nan mais uma vez e os substituindo por zero, e depois esses mesmos foram removidos.
deletar = df.loc[((df.Global_Sales.fillna(0)) == 0) & ((df.Total_Shipped.fillna(0)) == 0)]
a = deletar.index
dados = df.drop(a)

In [398]:
#juntando os as vendas em uma única categoria.
dados["Sales"] = dados["Global_Sales"].fillna(0) + dados["Total_Shipped"].fillna(0)

In [399]:
dados = dados.drop(columns = ["Global_Sales","Total_Shipped","url","status","img_url","NA_Sales" ,"PAL_Sales","JP_Sales" ,"Other_Sales","VGChartz_Score"])


In [400]:
limpa = dados.loc[((dados.Sales != 0 ) & (dados.Critic_Score != 0))]
limpa

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,Critic_Score,User_Score,Year,Last_Update,Vgchartzscore,Sales
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,7.7,0.0,2006.0,0,0.0,82.86
1,2,Super Mario Bros.,super-mario-bros,Platform,0,NES,Nintendo,Nintendo EAD,10.0,0.0,1985.0,0,0.0,40.24
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,8.2,9.1,2008.0,11th Apr 18,8.7,37.14
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,8.0,8.8,2009.0,0,8.8,33.09
5,6,Pokemon Red / Green / Blue Version,pokmon-red,Role-Playing,E,GB,Nintendo,Game Freak,9.4,0.0,1998.0,0,0.0,31.38
6,7,New Super Mario Bros.,new-super-mario-bros,Platform,E,DS,Nintendo,Nintendo EAD,9.1,8.1,2006.0,0,0.0,30.80
8,9,New Super Mario Bros. Wii,new-super-mario-bros-wii,Platform,E,Wii,Nintendo,Nintendo EAD,8.6,9.2,2009.0,0,9.1,30.22
9,10,Minecraft,minecraft,Misc,0,PC,Mojang,Mojang AB,10.0,0.0,2010.0,05th Aug 18,0.0,30.01
11,12,Wii Play,wii-play,Misc,E,Wii,Nintendo,Nintendo EAD,5.9,4.5,2007.0,0,5.3,28.02
12,13,Kinect Adventures!,kinect-adventures,Party,E,X360,Microsoft Game Studios,Good Science Studio,6.7,0.0,2010.0,29th Sep 18,6.2,24.00


**SGDRegressor**

In [401]:
#Importando a regressão.
from sklearn.linear_model import SGDRegressor

In [402]:
fitter = SGDRegressor(loss="squared_loss", penalty=None)

In [403]:
#utilizando a função dummify para converter palavras em numeros.
def dummify(data, column_name):
    """
        Converte a coluna column_name em dummies / one-hot e as adiciona ao dataframe
        retorna uma copia do  df original *sem* a coluna que foi dummified
    """
    df = data.copy()
    # Nota: se você for um econometrista, pode se interessar por usar a opção dropFirst = True na get_dummies
    df2 = pd.concat([df.drop(column_name, axis=1), pd.get_dummies(data[column_name], prefix=column_name)], axis=1)
    return df2

In [404]:
data_dum=dummify(limpa[["Critic_Score", "User_Score","Vgchartzscore","Genre","Sales","Platform","Publisher","Developer"]],["Genre","Platform","Publisher","Developer"])
len(data_dum.columns)

1408

In [405]:
lista_nova = list(data_dum.columns)
lista_nova.remove("Sales")
data_trei = data_dum[lista_nova]
sales = data_dum["Sales"]

In [492]:
#serparando o dataset em treinamento e tese
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_trei,sales, test_size=0.2, random_state=RANDOM_SEED)



In [493]:
X = X_train.as_matrix(columns=list(X_train.columns)) 


C:\Users\pedro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [494]:
X

array([[6.6, 0. , 0. , ..., 0. , 0. , 0. ],
       [7.7, 0. , 0. , ..., 0. , 0. , 0. ],
       [9. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [7.8, 0. , 0. , ..., 0. , 0. , 0. ],
       [7.3, 0. , 8. , ..., 0. , 0. , 0. ],
       [4.8, 4.8, 0. , ..., 0. , 0. , 0. ]])

In [495]:
y = pd.Series(y_train).tolist()

In [496]:
fitter.fit(X, y)

C:\Users\pedro\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=None,
       n_iter=None, n_iter_no_change=5, penalty=None, power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)

**Resultados da SGDRegressor**

In [497]:
resultado=fitter.predict(X_test)
resultado

array([ 1.32495229e+00,  1.71040544e+00,  1.72510891e+00, -1.47239530e-01,
        2.61388414e+00,  1.36427311e-01, -2.68149181e-01,  8.37407553e-01,
        2.69955525e-01,  1.19986809e+00,  1.05238309e-01,  2.63420656e-01,
        3.67308336e+00, -4.35020288e-01,  3.79381857e-01,  7.26689921e-01,
        7.07904139e-01,  5.84952369e-01,  6.19039165e-01,  2.37316727e+00,
        1.07762369e+00,  8.25620902e-01,  1.18461990e+00,  7.70625455e-01,
        8.86431218e-01,  1.12271823e+00,  4.39990836e-01,  9.74531242e-01,
        6.15559732e-01,  2.51444031e+00, -8.76620289e-01,  5.18977631e-01,
        2.07548979e+00,  3.78819236e-01,  8.63056277e-01, -1.83932687e-03,
        1.54295178e+00,  1.18793973e+00,  5.30943686e-01,  9.15595582e-01,
        1.49545733e+00,  1.33387948e+00,  3.59150849e+00,  8.76847467e-01,
        2.35375548e-01,  2.08479959e+00,  7.59406738e-01,  5.37751195e-01,
        1.92791599e+00,  2.57965415e+00,  7.94373350e-01,  9.91246819e-01,
        5.93004623e-01,  

In [498]:
from sklearn.metrics import mean_squared_error,r2_score,median_absolute_error,mean_absolute_error

In [499]:
mean_squared_error(y_test,resultado)

4.4906357737121745

In [500]:
r2_score(y_test,resultado)

0.24013700852243802

In [490]:
median_absolute_error(y_test,resultado)

1.247940739891564

In [491]:
mean_absolute_error(y_test,resultado)

1.510183010638453